# Our best model so far

Packages

In [71]:
import os 
import re

# data manipulation/viz
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

# modeling setups
from patsy import dmatrices
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.preprocessing import StandardScaler

# linear modeling
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import statsmodels.api as sm
from statsmodels.formula.api import glm 

# tree modeling
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# other
from sklearn.decomposition import PCA

# turn off the df['col'] = x assignment warning
#pd.options.mode.chained_assignment = None  # default='warn'

Load and data clean

In [72]:
# optional directory set-up
train = pd.read_csv("../../housing_data/train.csv")
test = pd.read_csv("../../housing_data/test.csv")
sample = pd.read_csv("../../housing_data/sample_submission.csv")

Data clean functions

In [73]:
def na_clean(df):
    
    # some vars are just too missing so I remove the field
    df = df.drop(columns = ["PoolQC", "MiscFeature"])

    # replace some numeric vars w/ median
    median_replace_vars = ['LotFrontage', 'MasVnrArea', 'GarageYrBlt', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageArea']
    for var in median_replace_vars:
        df[var].fillna(df[var].median(), inplace = True)
    
    # replace some num vars w/ 0
    zero_replace_vars = ['BsmtFullBath', 'BsmtHalfBath', 'GarageCars']
    for var in zero_replace_vars:
        df[var].fillna(0, inplace = True)
    
    # replace some cat vars w/ most freq value 
    df['MasVnrType'].fillna('None', inplace = True)
    df['Electrical'].fillna('SBrkr', inplace = True)
    df['MSZoning'].fillna('RL', inplace = True)
    df['SaleType'].fillna('WD', inplace = True)
    df['Utilities'].fillna('AllPub', inplace = True)
    df['KitchenQual'].fillna('TA', inplace = True)
    df['Functional'].fillna('Typ', inplace = True)

    # other cat vars just put missing if there isn't a glaring most popular category
    replace_missing_vars = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 
        'BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'Fence', 'Exterior1st', 
        'Exterior2nd', 'FireplaceQu']
    for var in replace_missing_vars:
        df[var].fillna("Missing", inplace = True)

    return df



# Function for some standard feature engineering to use in all models
def standard_feature_eng(df, test_data = False):
    '''Input either the training or test data. 
    2nd arg set to True if it's the testing data. That way we ignore the final log transformation on sale price'''

    # num features to just binarize b/c few houses have the feature
    df["SwimmingPool"] = df['PoolArea'].map(lambda x: 0 if x==0 else 1)
    df["3SsnPorch"] = df['3SsnPorch'].map(lambda x: 0 if x==0 else 1)
    df["ScreenPorch"] = df['ScreenPorch'].map(lambda x: 0 if x==0 else 1)

    # re-factoring vars:
    # group the irregularities into 2 factor levels
    df['LotShape'] = df['LotShape'].map({'Reg': 'Reg', 'IR1': 'Reg', 'IR2': 'Irreg', 'IR3': 'Irreg'})

    # simplifying MSSubClass because we have the year built in another feature
    df['MSSubClass'] = df['MSSubClass'].map(lambda x: 
        "1_story"   if (x in (20, 30, 40, 120)) else(
        "1.5_story" if (x in (45, 50, 150)) else(
        "2_story"   if (x in (60, 70, 75, 160, 180, 190)) else(
        "split"     if (x in (80, 85)) else(
        "duplex"    if (x ==90) else(
        "other"))))))
    df['MSSubClass'] = df['MSSubClass'].astype("object")

    # simplifying more vars
    # electrical:
    df['Electrical'] = df['Electrical'].map(lambda x: "SBrkr" if x == "SBrkr" else "Fuse")
    # exterior:
    df['Exterior'] = df['Exterior1st'].map(lambda x: 
        # group exterior into simplified var based on average prices
        "Expensive" if (x in ("VinylSd", "CemntBd", "Stone", "ImStucc")) else(
        "Cheap" if (x in ("BrkComm", "AsphShn", "CBlock", "AsbShng")) else(
        "Moderate")))
    df = df.drop(columns=['Exterior1st', 'Exterior2nd'])
    # garage
    df['GarageQual'] = df['GarageQual'].map(lambda x: 
        # group exterior into simplified var based on average prices
        "Good" if (x in ("Ex", "Gd")) else(
        "Medium" if (x in ("TA")) else(
        "Bad")))
    df['Heating'] = df['Heating'].map(lambda x: "Gas" if x in ("GasA", "GasW") else "Other")

    # deciding to drop a few features for various reasons
    vars_to_drop = [
        # not much variation:
        "LowQualFinSF", 
        "LandSlope", 
        "MiscVal", 
        "RoofMatl",
        "Condition2",
        #"KitchenAbvGr" # hardly any variation. But, Deva included in lm's so including it now.
        "PoolArea", # binarized above
        "Utilities", # only 1 obs in training data different from regular
        "HouseStyle" # already explained in other vars
        ]
    df.drop(columns=vars_to_drop, inplace=True) 

    # adding a remodeled feature
    df['Remodeled'] = (df.YearRemodAdd-df.YearBuilt) == 0

    # total inside area will be a sum of 1st and 2nd floor sq ft
    df['Total_Inside_Area'] = df['1stFlrSF'] + df['2ndFlrSF']
    df.drop(columns = ['1stFlrSF', '2ndFlrSF', 'GrLivArea'], inplace = True)

    # simplify the bathrooms variable
    df['Bathrooms'] = df.BsmtFullBath + 0.5*df.BsmtHalfBath + df.FullBath + 0.5*df.HalfBath
    df.drop(columns = ['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], inplace = True)

    # get log of sale price which will be our actual response variable
    if test_data:
        pass 
    else:
        df['LogSalePrice'] = np.log(df.SalePrice)

    return df


OLS with 16 features predicting regular sale price

In [74]:
def lm_df_clean(df, test_data = False):

    # first run standard data cleaning steps
    df = na_clean(df)
    df = standard_feature_eng(df, test_data = test_data)

    lm_vars = ['LotArea', 'Street', 'Neighborhood', 'OverallQual', 'OverallCond', 'YearRemodAdd', 
              'BsmtCond', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'YrSold', 
              'MoSold', 'Remodeled', 'Total_Inside_Area', 'Bathrooms']

    df = pd.get_dummies(df[lm_vars], 
        columns = ['Street', 'Neighborhood', 'OverallQual', 'OverallCond', 'BsmtCond','KitchenQual'], 
        drop_first=True)
    
    return df


# data setups
X_train = lm_df_clean(train)
X_test = lm_df_clean(test, test_data=True)
Y_train = train.SalePrice
Y_test = sample.SalePrice

# fit to train data
lr_1 = LinearRegression(fit_intercept=True).fit(X_train, Y_train)

# evaluate performance
print("Score on training data: {:.3f}".format(lr_1.score(X_train,Y_train)))
print("Score on testing data: {:.3f}".format(lr_1.score(X_test ,Y_test)))

yhat_train = lr_1.predict(X_train)
yhat_test = lr_1.predict(X_test)

# rmse
rmse_train = (np.mean((np.log(yhat_train) - np.log(Y_train))**2))**.5
rmse_test = (np.mean((np.log(yhat_test) - np.log(Y_test))**2))**.5

print("Log RMSE on training data: {:.3f}".format(rmse_train))
print("Log RMSE on testing data: {:.3f}".format(rmse_test))

Score on training data: 0.850
Score on testing data: -17.593
Log RMSE on training data: 0.146
Log RMSE on testing data: 0.378


OLS with 1 variable (overall quality) predicting regular sale price

In [75]:
# data setup function
def lm_overall_quality_df_clean(df, test_data = False):

    # first run standard data cleaning steps
    df = na_clean(df)
    df = standard_feature_eng(df, test_data = test_data)
    df = df.loc[:, ['OverallQual']]
    
    return df

# data setups
X_train = lm_overall_quality_df_clean(train)
X_test = lm_overall_quality_df_clean(test, test_data=True)
Y_train = train.SalePrice
Y_test = sample.SalePrice

# fit to train data
lr_overall_quality = LinearRegression(fit_intercept=True).fit(X_train, Y_train)

# Evaluate performance
print("Score on training data: {:.3f}".format(lr_overall_quality.score(X_train,Y_train)))
print("Score on testing data: {:.3f}".format(lr_overall_quality.score(X_test ,Y_test)))

yhat_train = lr_overall_quality.predict(X_train)
yhat_test = lr_overall_quality.predict(X_test)

# set negative values to 0.1
yhat_train = np.array([0.1 if i < 0 else i for i in yhat_train])
yhat_test = [0.1 if i < 0 else i for i in yhat_test]

# rmse
rmse_train = (np.mean((np.log(yhat_train) - np.log(Y_train))**2))**.5
rmse_test = (np.mean((np.log(yhat_test) - np.log(Y_test))**2))**.5

print("Log RMSE on training data: {:.3f}".format(rmse_train))
print("Log RMSE on testing data: {:.3f}".format(rmse_test))

Score on training data: 0.626
Score on testing data: -14.882
Log RMSE on training data: 0.811
Log RMSE on testing data: 1.354


Lasso predicting log sale price

In [76]:
# first build one hot encoder based on the training data
train_lasso = standard_feature_eng(na_clean(train))
enc_lasso = OneHotEncoder(handle_unknown = 'ignore')
enc_lasso.fit(train_lasso.select_dtypes(include=["object"]))
one_hot_columns = pd.get_dummies(train_lasso.select_dtypes(include=["object"])).columns
# will use this encoder in the function below

# data setup function
def lasso_df_clean(df, test_data = False):

    # first run standard data cleaning steps
    df = na_clean(df)
    df = standard_feature_eng(df, test_data = test_data)

    # one hot encode using encoder above
    categorical_cols = pd.DataFrame(enc_lasso.transform(df.select_dtypes(include=["object"])).toarray())
    categorical_cols.columns = one_hot_columns
    df = pd.concat([categorical_cols, df.select_dtypes(exclude=["object"])], axis=1)

    # log transformations
    #df["GrLivArea"] = np.log(df["GrLivArea"])
    
    # select only vars needed
    if test_data:
        df = df.drop(columns=["Id"])
    else:
        df = df.drop(columns=["Id"])
        df['SalePrice'] = np.log(df['SalePrice'])
        #df = df[["GrLivArea","OverallQual", "SalePrice"]]
    
    return df


# data setups
X_train = lasso_df_clean(train)
X_test = lasso_df_clean(test, test_data=True)
Y_train = X_train.SalePrice
X_train = X_train.drop(columns=['LogSalePrice', 'SalePrice'])
Y_test = np.log(sample.SalePrice)

# fit to train data
lasso_fit = Lasso(alpha=.01).fit(X_train, Y_train)

# Evaluate performance
yhat_train = lasso_fit.predict(X_train)
yhat_test = lasso_fit.predict(X_test)

# Evaluate performance
print("Score on training data: {:.3f}".format(lasso_fit.score(X_train,Y_train)))
print("Score on testing data: {:.3f}".format(lasso_fit.score(X_test ,Y_test)))

# rmse
rmse_train = (np.mean((yhat_train - np.log(train.SalePrice))**2))**.5
rmse_test = (np.mean((yhat_test - np.log(sample.SalePrice))**2))**.5

print("Log RMSE on training data: {:.3f}".format(rmse_train))
print("Log RMSE on testing data: {:.3f}".format(rmse_test))

Score on training data: 0.850
Score on testing data: -14.746
Log RMSE on training data: 0.155
Log RMSE on testing data: 0.357


Random Forest to predict log sale price

In [77]:
# first build one hot encoder based on the training data
train_rf = standard_feature_eng(na_clean(train))
enc_rf = OneHotEncoder(handle_unknown = 'ignore')
enc_rf.fit(train_rf.select_dtypes(include=["object"]))
one_hot_columns = pd.get_dummies(train_rf.select_dtypes(include=["object"])).columns
# will use this encoder in the function below

# Random forest data clean function
def rf_df_clean(df, test_data = False):

    # first run standard data cleaning steps
    df = na_clean(df)
    df = standard_feature_eng(df, test_data = test_data)

    # one hot encode using encoder above
    categorical_cols = pd.DataFrame(enc_rf.transform(df.select_dtypes(include=["object"])).toarray())
    categorical_cols.columns = one_hot_columns
    df = pd.concat([categorical_cols, df.select_dtypes(exclude=["object"])], axis=1)
    
    # DO FEATURE ENGINEERING HERE
    # drop irrelevant columns
    df = df.drop(columns=["Id"])
    
    return df


# preprocess the data
df_rf = rf_df_clean(train)
df_rf_test = rf_df_clean(test, test_data=True)

# run model on best parameters
rf_reg = RandomForestRegressor(
    n_estimators = 900,
    max_depth = 25,
    max_features = 'auto',
    min_samples_split = 2,  
    bootstrap = True, 
    )

# fit the model
rf_reg = rf_reg.fit(df_rf.drop(columns = ["SalePrice", 'LogSalePrice']), df_rf.LogSalePrice)

# Evaluate performance
yhat_train = rf_reg.predict(df_rf.drop(columns = ["SalePrice", 'LogSalePrice']))
yhat_test = rf_reg.predict(df_rf_test)

# Evaluate performance
print("Score on training data: {:.3f}".format(rf_reg.score(df_rf.drop(columns = ["SalePrice", 'LogSalePrice']), df_rf.LogSalePrice)))
print("Score on testing data: {:.3f}".format(rf_reg.score(df_rf_test, np.log(sample.SalePrice))))

# rmse
rmse_train = (np.mean((yhat_train - np.log(train.SalePrice))**2))**.5
rmse_test = (np.mean((yhat_test - np.log(sample.SalePrice))**2))**.5

print("Log RMSE on training data: {:.3f}".format(rmse_train))
print("Log RMSE on testing data: {:.3f}".format(rmse_test))

Score on training data: 0.983
Score on testing data: -15.389
Log RMSE on training data: 0.052
Log RMSE on testing data: 0.364


XGBoost to predict log sale price

In [78]:
# first build one hot encoder based on the training data
train_xgb = standard_feature_eng(na_clean(train))
enc_xgb = OneHotEncoder(handle_unknown = 'ignore')
enc_xgb.fit(train_xgb.select_dtypes(include=["object"]))
one_hot_columns_xgb = pd.get_dummies(train_xgb.select_dtypes(include=["object"])).columns
# will use this encoder in the function below

# xgboost data clean function
def xgb_df_clean(df, test_data = False):

    # first run standard data cleaning steps
    df = na_clean(df)
    df = standard_feature_eng(df, test_data = test_data)

    # one hot encode using encoder above
    categorical_cols = pd.DataFrame(enc_xgb.transform(df.select_dtypes(include=["object"])).toarray())
    categorical_cols.columns = one_hot_columns_xgb
    df = pd.concat([categorical_cols, df.select_dtypes(exclude=["object"])], axis=1)
    
    # DO MORE FEATURE ENGINEERING HERE LATER
    df = df.drop(columns = ['Id'])
    
    return df


# preprocess the data
df_xgb = xgb_df_clean(train)
df_xgb_test = xgb_df_clean(test, test_data=True)

# get X feature names
xgb_cols = np.array(df_xgb.drop(columns = ["SalePrice", 'LogSalePrice']).columns)

# convert data to DMatrix format
dmat_train = xgb.DMatrix(df_xgb.drop(columns = ["SalePrice", 'LogSalePrice']), df_xgb['LogSalePrice'], feature_names=xgb_cols)
dmat_test = xgb.DMatrix(df_xgb_test, np.log(sample.SalePrice), feature_names=xgb_cols)

# train model
booster = xgb.train({

    "booster": "gbtree", 
    "max_depth": 30, 
    "eta": .4, 
    "gamma": .01, 
    "subsample": 0.6,
    "lambda": .7, 
    "alpha": 0, 
    "max_bin": 256, 
    "colsample_bytree": .7, # proportion of features
    "eval_metric": "rmse", 
    "objective": "reg:squarederror"
    },

    dmat_train,
    evals=[(dmat_train, "train"), (dmat_test, "test")] 
)

# Evaluate performance
yhat_train = booster.predict(dmat_train)
yhat_test = booster.predict(dmat_test)

# # rmse
rmse_train = (np.mean((yhat_train - np.log(train.SalePrice))**2))**.5
rmse_test = (np.mean((yhat_test - np.log(sample.SalePrice))**2))**.5

print("\nLog RMSE on training data: {:.3f}".format(rmse_train))
print("Log RMSE on testing data: {:.3f}".format(rmse_test))

[0]	train-rmse:6.93301	test-rmse:6.99010
[1]	train-rmse:4.17002	test-rmse:4.23007
[2]	train-rmse:2.51226	test-rmse:2.57474
[3]	train-rmse:1.52339	test-rmse:1.59279
[4]	train-rmse:0.92837	test-rmse:1.01119
[5]	train-rmse:0.57398	test-rmse:0.68744
[6]	train-rmse:0.36331	test-rmse:0.51933
[7]	train-rmse:0.24082	test-rmse:0.44865
[8]	train-rmse:0.16872	test-rmse:0.41735
[9]	train-rmse:0.12559	test-rmse:0.40361

Log RMSE on training data: 0.126
Log RMSE on testing data: 0.404


Final Ensemble model

In [79]:
def housing_ensemble_model(df):
    '''This is the final model that takes in raw data, and makes predictions'''

    # OLS
    yhat_ols1 = lr_1.predict(lm_df_clean(df, test_data=True))

    # OLS Simple
    yhat_ols_simple = lr_overall_quality.predict(lm_overall_quality_df_clean(df, test_data=True))

    # GLM

    # Lasso
    lasso_data = lasso_df_clean(df, test_data=True)
    yhat_lasso = np.exp(lasso_fit.predict(lasso_data))

    # Random forest
    yhat_rf = np.exp(rf_reg.predict(rf_df_clean(df, test_data=True)))

    # XGB
    xgb_data = xgb.DMatrix(xgb_df_clean(df, test_data=True)) 
    yhat_xgb = np.exp(booster.predict(xgb_data))

    # make ensemble prediction
    # Do more work to come up w/ weights. This is just a sample
    yhat_final = yhat_ols1*(1/5) + yhat_ols_simple * (1/5) + yhat_lasso * (1/5) + yhat_rf * (1/5) + yhat_xgb * (1/5)

    return yhat_final


# final ensemble model RMSE
yhat_train = housing_ensemble_model(train.drop(columns = ["SalePrice"]))
yhat_test = housing_ensemble_model(test)

rmse_train = np.mean((train.SalePrice - yhat_train)**2)**.5
rmse_test = np.mean((sample.SalePrice - yhat_test)**2)**.5

# evaluate rmse  on the testing data
print("RMSE Train: {}".format(rmse_train))
print("RMSE Test: {}".format(rmse_test))

# log results
rmse_train_log = np.mean((np.log(train.SalePrice) - np.log(yhat_train))**2)**.5
rmse_test_log = np.mean((np.log(sample.SalePrice) - np.log(yhat_test))**2)**.5
# evaluate rmse  on the testing data
print("\nLog RMSE Train: {}".format(rmse_train_log))
print("Log RMSE Test: {}".format(rmse_test_log))

RMSE Train: 28279.128974825577
RMSE Test: 65377.90673992918

Log RMSE Train: 0.12183455865475633
Log RMSE Test: 0.3655684085054176


Create sample submission

In [80]:
sample_submission = pd.DataFrame({
    "Id": test.Id,
    "SalePrice": yhat_test
})
sample_submission.to_csv("~/Desktop/sample_submission_11-13-2021.csv", index=False)

Kaggle Score: 0.14089

Rank: 2042 / 4683 = 43.6%